In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext

In [6]:
file_path1 = r"C:\Users\ejian\CogWorks-2022-Gausslien-Final-Capstone\Sarcasm Detector\Sarcasm_Headlines_Dataset.json"
df = pd.read_json(file_path1,lines=True)
df = df[['headline','is_sarcastic']]
#print(df[:50])

In [150]:
df

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


In [88]:
x = np.asarray(df)

array(["former versace store clerk sues over secret 'black code' for minority shoppers",
       0, 0], dtype=object)

In [91]:
x.shape

(26709, 2)

In [93]:
x[:,1]

array([0, 0, 1, ..., 0, 0, 0], dtype=object)

In [71]:
df[23793]

KeyError: 23793

In [97]:
import random
import numpy as np

def train_split(data, validation=0.2):
    N = len(data)
    print(N)
    breakpoint = int(validation*N)
    
    data_idxs = list(np.arange(N))
    
    random.shuffle(data_idxs)
    #print(data_idxs)
    
    return data[data_idxs[:breakpoint], :], data[data_idxs[breakpoint:], :]

In [100]:
testing_data, training_data = train_split(x)

26709


In [108]:
testing_data.shape
testing_data

array([['uc davis chancellor linda katehi placed on leave over claims of ethics violations',
        0],
       ['unmanned military drone briefly grasps senselessness of war', 1],
       ["this 1927 essay proves we've always worried about the future of books",
        0],
       ...,
       ['tough-guy ice agent struggling to raise adorable kids after deporting their parents',
        1],
       ["the corruption beneath cuomo's casino push", 0],
       ['new york times throws vicious oscars shade at kevin spacey', 0]],
      dtype=object)

In [103]:
training_data.shape

(21368, 2)

In [109]:
#tokenizer stuff
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")

def tokenize(data):
    tokens = [tokenizer(headline) for headline in data[:, 0]]
    return tokens

In [146]:
tokenizedTestData = tokenize(training_data)

In [147]:
vocab = torchtext.vocab.build_vocab_from_iterator(tokenizedTestData)

In [148]:
vocabArr = np.array([word for word in vocab.get_stoi().keys()])

In [149]:
len(vocabArr)

24966

In [10]:
class Model(nn.Module):
    def __init__(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim, pad_idx = pad_idx)
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim, n_layers = n_layers, dropout = dropout, bidirectional = True)
        self.Linear = nn.Linear(hidden_dim, output_dim)
#         self.Dropout = nn.dropout(dropout = dropout)
        
    def forward(self, text, text_length):
        embedding = self.embed(text)
        
        packedEmbeddings = torch.nn.utils.rnn.pack_padded_sequence(text, text_length)
        packedOutput, (hidden, cell) = self.rnn(packedEmbeddings)
        unpackedOutput, unpackedLength = pad_packed_sequence(packedOutput)
        
        return self.Linear(hidden)
 

In [155]:
vocab_size = len(vocabArr)
embedding_dim = 200
hidden_dim = 10
output_dim = 1
n_layers = 2
dropout = 0.5
pad_idx = 0

model = {vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx}